In [2]:
import torchaudio
import torch
from demucs.pretrained import get_model
from demucs.apply import apply_model
import os

# Create output directory
output_dir = '../outputs'
os.makedirs(output_dir, exist_ok=True)
song_path = '../Zach_Bryan_-_Something_In_The_Orange.mp3'
model_name = 'htdemucs_6s'
song_name = os.path.splitext(song_path)[0].replace('_', '')  # Correctly extract filename without extension
os.makedirs(f'{output_dir}/{song_name}', exist_ok=True)

# Load the audio
sample_waveform, sample_rate = torchaudio.load(song_path)
print(f"Loaded audio with shape {sample_waveform.shape} and sample rate {sample_rate}")

# Load the model
# model = get_model("htdemucs_6s")
model = get_model(model_name)
model.eval()

# Check for device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
print(f"Using device: {device}")
model.to(device)

# Ensure correct format
if sample_rate != model.samplerate:
    waveform = torchaudio.functional.resample(sample_waveform, sample_rate, model.samplerate)
    print(f"Resampled from {sample_rate} to {model.samplerate}")
else:
    waveform = sample_waveform

# Handle channels if needed
if waveform.shape[0] > 2:
    waveform = waveform[:2, :]
elif waveform.shape[0] == 1:
    waveform = torch.cat([waveform, waveform], dim=0)

waveform = waveform.to(device)

def enhance_guitar(guitar_waveform, sample_rate):
    # Add batch dimension if needed
    if guitar_waveform.dim() == 2:
        guitar_waveform = guitar_waveform.unsqueeze(0)
        
    # Now we can safely unpack dimensions
    b, c, t = guitar_waveform.shape
    
    guitar_waveform_freq = torch.fft.rfft(guitar_waveform, dim=2)
    
    # Create high-pass filter (reduce below 80Hz)
    freqs = torch.fft.rfftfreq(t, d=1/sample_rate)
    high_pass = (1 - torch.exp(-freqs/80))
    
    # Apply filter
    guitar_waveform_freq *= high_pass.view(1, 1, -1)
    guitar_waveform = torch.fft.irfft(guitar_waveform_freq, n=t, dim=2)
    
    # Apply subtle compression
    peak = guitar_waveform.abs().max()
    if peak > 0:
        guitar_waveform = 0.9 * guitar_waveform / peak
    
    # Remove batch dimension if we added it
    if b == 1:
        guitar_waveform = guitar_waveform.squeeze(0)
        
    return guitar_waveform

# Separate
print("Separating sources...")
with torch.no_grad():
    sources = apply_model(model, waveform.unsqueeze(0))[0]
    sources = sources.cpu()

# Print available sources
print(f"Model sources: {model.sources}")

# Save all sources
for idx, source_name in enumerate(model.sources):
    source_wav = sources[idx]
    output_file = os.path.join(output_dir, song_name, f"separated_{source_name}.wav")
    
    # Save original source
    torchaudio.save(output_file, source_wav, model.samplerate)
    print(f"Saved {source_name} to {output_file}")
    

    # For guitar, also save enhanced version
    if source_name == 'guitar':
        enhanced_guitar_wav = enhance_guitar(source_wav, model.samplerate)
        output_file_enhanced = os.path.join(output_dir, song_name, f"separated_{source_name}_enhanced.wav")
        torchaudio.save(output_file_enhanced, enhanced_guitar_wav, model.samplerate)
        print(f"Saved enhanced {source_name} to {output_file_enhanced}")

Loaded audio with shape torch.Size([2, 12602306]) and sample rate 48000
Using device: mps
Resampled from 48000 to 44100
Separating sources...
Model sources: ['drums', 'bass', 'other', 'vocals', 'guitar', 'piano']
Saved drums to ../outputs/../ZachBryan-SomethingInTheOrange/separated_drums.wav
Saved bass to ../outputs/../ZachBryan-SomethingInTheOrange/separated_bass.wav
Saved other to ../outputs/../ZachBryan-SomethingInTheOrange/separated_other.wav
Saved vocals to ../outputs/../ZachBryan-SomethingInTheOrange/separated_vocals.wav
Saved guitar to ../outputs/../ZachBryan-SomethingInTheOrange/separated_guitar.wav
Saved enhanced guitar to ../outputs/../ZachBryan-SomethingInTheOrange/separated_guitar_enhanced.wav
Saved piano to ../outputs/../ZachBryan-SomethingInTheOrange/separated_piano.wav


In [20]:
model.sources

['drums', 'bass', 'other', 'vocals', 'guitar', 'piano']

In [27]:
import torchaudio
import torch
from demucs.pretrained import get_model
from demucs.apply import apply_model
import os

# Create output directory
output_dir = 'outputs'
os.makedirs(output_dir, exist_ok=True)
song_path = 'Bon_Iver_St._Vincent_-_Roslyn_Lyrics.mp3'
song_name = os.path.splitext(os.path.basename(song_path))[0].replace('_', '')
os.makedirs(f'{output_dir}/{song_name}', exist_ok=True)

# Load the audio
print(f"Loading audio from {song_path}...")
sample_waveform, sample_rate = torchaudio.load(song_path)
print(f"Loaded audio with shape {sample_waveform.shape} and sample rate {sample_rate}")

# Determine device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

def prepare_audio(waveform, source_sr, target_sr):
    """Prepare audio for model input"""
    # Resample if needed
    if source_sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, source_sr, target_sr)
        
    # Handle channels
    if waveform.shape[0] > 2:
        waveform = waveform[:2, :]
    elif waveform.shape[0] == 1:
        waveform = torch.cat([waveform, waveform], dim=0)
        
    return waveform

def enhance_guitar(guitar_waveform, sample_rate):
    """Enhance guitar with filters and transient processing"""
    # Add batch dimension if needed
    if guitar_waveform.dim() == 2:
        guitar_waveform = guitar_waveform.unsqueeze(0)
        
    # Now we can safely unpack dimensions
    b, c, t = guitar_waveform.shape
    
    # FFT for frequency domain processing
    guitar_waveform_freq = torch.fft.rfft(guitar_waveform, dim=2)
    
    # Create high-pass filter (reduce below 80Hz)
    freqs = torch.fft.rfftfreq(t, d=1/sample_rate)
    high_pass = (1 - torch.exp(-freqs/80))
    
    # Create mid boost around 2-4kHz (presence)
    mid_boost = 1.0 + 0.5 * torch.exp(-((freqs - 3000)/500)**2)
    
    # Apply filters
    filter_curve = high_pass.view(1, 1, -1) * mid_boost.view(1, 1, -1)
    guitar_waveform_freq *= filter_curve
    
    # Back to time domain
    guitar_waveform = torch.fft.irfft(guitar_waveform_freq, n=t, dim=2)
    
    # Apply subtle compression
    peak = guitar_waveform.abs().max()
    if peak > 0:
        # Simple soft knee compression
        threshold = 0.7
        ratio = 3.0
        gain = 1.2
        
        above_thresh = (guitar_waveform.abs() > threshold * peak).float()
        comp_factor = 1.0 - above_thresh * (1.0 - 1.0/ratio) * (guitar_waveform.abs() - threshold * peak) / (peak * (1.0 - threshold))
        guitar_waveform = guitar_waveform * comp_factor * gain
        
        # Final limiter
        peak = guitar_waveform.abs().max()
        if peak > 0.95:
            guitar_waveform = 0.95 * guitar_waveform / peak
    
    # Remove batch dimension if we added it
    if b == 1:
        guitar_waveform = guitar_waveform.squeeze(0)
        
    return guitar_waveform

# STAGE 1: Extract all stems with htdemucs_ft
print("STAGE 1: Separating with htdemucs_ft...")
model_stage1 = get_model("htdemucs_ft")
model_stage1.eval()
model_stage1.to(device)

# Prepare audio for first model
waveform_stage1 = prepare_audio(sample_waveform, sample_rate, model_stage1.samplerate)
waveform_stage1 = waveform_stage1.to(device)

# Separate first stage
with torch.no_grad():
    sources_stage1 = apply_model(model_stage1, waveform_stage1.unsqueeze(0))[0]
    sources_stage1 = sources_stage1.cpu()

# Get the "other" stem
other_index = model_stage1.sources.index('other') if 'other' in model_stage1.sources else None
if other_index is None:
    print("Warning: 'other' source not found in model 1. Using all non-guitar sources combined.")
    # Combine all sources except guitar to create "other"
    if 'guitar' in model_stage1.sources:
        guitar_index = model_stage1.sources.index('guitar')
        all_sources = torch.zeros_like(sources_stage1[0])
        for i, src in enumerate(model_stage1.sources):
            if i != guitar_index:
                all_sources += sources_stage1[i]
        other_waveform = all_sources
    else:
        # If no guitar source, just use the first stem as "other"
        other_waveform = sources_stage1[0]
else:
    other_waveform = sources_stage1[other_index]

# Save the other stem
other_file = os.path.join(output_dir, song_name, "stage1_other.wav")
torchaudio.save(other_file, other_waveform, model_stage1.samplerate)
print(f"Saved 'other' stem to {other_file}")

# STAGE 2: Extract guitar from "other" stem using htdemucs_6s
print("STAGE 2: Extracting guitar from 'other' using htdemucs_6s...")
model_stage2 = get_model("htdemucs_6s")
model_stage2.eval()
model_stage2.to(device)

# Prepare the "other" stem for second model
other_waveform = prepare_audio(other_waveform, model_stage1.samplerate, model_stage2.samplerate)
other_waveform = other_waveform.to(device)

# Separate second stage
with torch.no_grad():
    sources_stage2 = apply_model(model_stage2, other_waveform.unsqueeze(0))[0]
    sources_stage2 = sources_stage2.cpu()

# Get the guitar from second separation
if 'guitar' in model_stage2.sources:
    guitar_index = model_stage2.sources.index('guitar')
    extracted_guitar = sources_stage2[guitar_index]
    
    # Save the extracted guitar
    guitar_file = os.path.join(output_dir, song_name, "stage2_guitar_from_other.wav")
    torchaudio.save(guitar_file, extracted_guitar, model_stage2.samplerate)
    print(f"Saved extracted guitar to {guitar_file}")
    
    # Enhance and save
    enhanced_guitar = enhance_guitar(extracted_guitar, model_stage2.samplerate)
    enhanced_file = os.path.join(output_dir, song_name, "stage2_guitar_enhanced.wav")
    torchaudio.save(enhanced_file, enhanced_guitar, model_stage2.samplerate)
    print(f"Saved enhanced guitar to {enhanced_file}")
else:
    print("Error: 'guitar' source not found in the second model")

# BONUS: Also get the guitar from the first separation for comparison
if 'guitar' in model_stage1.sources:
    guitar_index = model_stage1.sources.index('guitar')
    original_guitar = sources_stage1[guitar_index]
    
    # Save the original guitar stem
    orig_guitar_file = os.path.join(output_dir, song_name, "stage1_original_guitar.wav")
    torchaudio.save(orig_guitar_file, original_guitar, model_stage1.samplerate)
    print(f"Saved original guitar stem to {orig_guitar_file}")
    
    # Create an enhanced version of the original guitar
    enhanced_orig_guitar = enhance_guitar(original_guitar, model_stage1.samplerate)
    enhanced_orig_file = os.path.join(output_dir, song_name, "stage1_original_guitar_enhanced.wav")
    torchaudio.save(enhanced_orig_file, enhanced_orig_guitar, model_stage1.samplerate)
    print(f"Saved enhanced original guitar to {enhanced_orig_file}")
    
    # FINAL STEP: Try combining both guitar extractions for maximum clarity
    # Resample if needed to match sample rates
    if model_stage1.samplerate != model_stage2.samplerate:
        original_guitar = torchaudio.functional.resample(
            original_guitar, model_stage1.samplerate, model_stage2.samplerate)
    
    # Make sure shapes match
    min_length = min(original_guitar.shape[1], extracted_guitar.shape[1])
    original_guitar = original_guitar[:, :min_length]
    extracted_guitar = extracted_guitar[:, :min_length]
    
    # Blend with 70% from first model, 30% from second model
    combined_guitar = 0.7 * original_guitar + 0.3 * extracted_guitar
    
    # Enhance the combined result
    enhanced_combined = enhance_guitar(combined_guitar, model_stage2.samplerate)
    combined_file = os.path.join(output_dir, song_name, "combined_guitar_enhanced.wav")
    torchaudio.save(combined_file, enhanced_combined, model_stage2.samplerate)
    print(f"Saved combined enhanced guitar to {combined_file}")

print("Processing complete!")

Loading audio from Bon_Iver_St._Vincent_-_Roslyn_Lyrics.mp3...
Loaded audio with shape torch.Size([2, 14800214]) and sample rate 48000
Using device: mps
STAGE 1: Separating with htdemucs_ft...
Saved 'other' stem to outputs/BonIverSt.Vincent-RoslynLyrics/stage1_other.wav
STAGE 2: Extracting guitar from 'other' using htdemucs_6s...
Saved extracted guitar to outputs/BonIverSt.Vincent-RoslynLyrics/stage2_guitar_from_other.wav
Saved enhanced guitar to outputs/BonIverSt.Vincent-RoslynLyrics/stage2_guitar_enhanced.wav
Processing complete!


## Analysis

Audio File → Audio Analysis Model → Extract tempo/rhythm → 
Text Description → LLM → Strumming Suggestions

In [23]:
import torchaudio

def trim_audio(input_file, output_file=None, start_sec=0, end_sec=None):
    """
    Trim audio file to specified start and end times.
    
    Parameters:
    - input_file: Path to the input audio file
    - output_file: Path to save the trimmed file (if None, returns without saving)
    - start_sec: Start time in seconds
    - end_sec: End time in seconds (if None, trims to the end of the file)
    
    Returns:
    - trimmed_waveform: Tensor containing the trimmed audio
    - sample_rate: Sample rate of the audio
    """
    # Load the audio
    waveform, sample_rate = torchaudio.load(input_file)
    
    # Convert time to samples
    start_sample = int(start_sec * sample_rate)
    end_sample = int(end_sec * sample_rate) if end_sec is not None else waveform.shape[1]
    
    # Trim the audio
    trimmed_waveform = waveform[:, start_sample:end_sample]
    
    # Save the trimmed audio if output_file is provided
    if output_file:
        torchaudio.save(output_file, trimmed_waveform, sample_rate)
    
    return trimmed_waveform, sample_rate

trim_audio('outputs/JeffBuckley-LoverYouShouldveComeOverAudio/stage2_guitar_enhanced.wav', start_sec = 45, end_sec = 100, output_file='outputs/JeffBuckley-LoverYouShouldveComeOverAudio/stage2_guitar_enhanced_cut.wav')

(tensor([[-0.0122, -0.0032,  0.0040,  ..., -0.0447, -0.0525, -0.0599],
         [ 0.0931,  0.1017,  0.0993,  ...,  0.0392,  0.0278,  0.0181]]),
 44100)

In [28]:
"""dynamic_guitar_strum_analysis.py – Notebook‑friendly version  (patched for librosa ≥ 1.0)
================================================================================

### 2025‑05‑03 patch
* **Fixed** “TypeError: chroma_cqt() takes 0 positional arguments …” that appears with
  librosa ≥ 1.0, which enforces *keyword‑only* arguments.  The call is now
  `chroma_cqt(y=y_harm, sr=sr)`.
* All other librosa calls remain version‑portable (tested 0.9 → 1.0).

Copy this entire cell (or keep as `.py` module) and run:
```python
from dynamic_guitar_strum_analysis import run_in_notebook
run_in_notebook('my_track.wav')
```
"""
from __future__ import annotations
import warnings, json, math, itertools, sys
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple

import numpy as np
import librosa
import pandas as pd
from tqdm import tqdm

# -----------------------------------------------------------------------------
# Dataclasses
# -----------------------------------------------------------------------------
@dataclass
class Strum:
    time: float         # seconds
    bar: int            # 1‑based measure number
    sub_16: int         # 0‑15 inside bar (16‑note grid)
    direction: str      # "D" or "U"
    velocity: float     # RMS energy normalised 0‑1
    kind: str           # "NOTE" | "CHORD" | "NONE"
    label: str          # e.g. "A#", "Dm", "N"

@dataclass
class BarSummary:
    bar: int
    bit_pattern: str       # 16 chars 0/1
    down_up: str           # 16 chars D/U/-
    mean_vel: float
    chords: List[str]

@dataclass
class Section:
    start_bar: int
    end_bar: int
    pattern_bits: str
    chords: List[str]

# -----------------------------------------------------------------------------
# Helper functions
# -----------------------------------------------------------------------------

def spectral_centroid_direction(y: np.ndarray, sr: int, onset_frames: np.ndarray) -> List[str]:
    """Classify Down/Up strokes by sign of spectral‑centroid slope around attack."""
    cent = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=512)[0]
    dirs = []
    for f in onset_frames:
        a = max(0, f-2)
        b = min(len(cent)-1, f+2)
        dirs.append('D' if cent[b] - cent[a] < 0 else 'U')
    return dirs


def chord_sequence_madmom(audio_path: str, sr: int) -> Dict[int, str]:
    """Return dict {beat_index: chord_label}.  If madmom missing, fallback returns empty dict."""
    try:
        from madmom.features.chords import CNNChordRecognitionProcessor
        proc = CNNChordRecognitionProcessor()
        chords = proc(audio_path)  # array [ onset, offset, label ]
        # Quantise onsets to beat indices
        y, _ = librosa.load(audio_path, sr=sr)
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr, units='frames')
        beat_times = librosa.frames_to_time(beats, sr=sr)
        out = {}
        for onset, _, lab in chords:
            idx = int(np.argmin(np.abs(beat_times - onset)))
            out[idx] = lab.split('/')[0]  # remove inversions "C/E" → "C"
        return out
    except Exception as e:
        warnings.warn("madmom not available – chord labels will be rough.")
        return {}


def label_note_or_chord(chroma: np.ndarray, frame: int, note_names=None) -> Tuple[str, str]:
    """Return (kind, label) where kind ∈ {NOTE, CHORD, NONE}"""
    if note_names is None:
        note_names = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
    frame = max(0, min(chroma.shape[1]-1, frame))
    v = chroma[:, frame]
    if v.max() < 1e-6:
        return 'NONE', 'N'
    thresh = 0.45 * v.max()
    strong = np.where(v >= thresh)[0]
    if len(strong) == 1:
        return 'NOTE', note_names[strong[0]]
    # chord heuristic
    root = strong[0]
    intervals = {(n-root) % 12 for n in strong}
    if {4,7}.issubset(intervals):
        qual = 'maj'
    elif {3,7}.issubset(intervals):
        qual = 'min'
    else:
        qual = ''
    return 'CHORD', f"{note_names[root]}{qual}"


def levenshtein(a: str, b: str) -> int:
    import Levenshtein
    return Levenshtein.distance(a, b)

# -----------------------------------------------------------------------------
# Main analysis function
# -----------------------------------------------------------------------------

def analyse_audio(audio_path: str, return_dataframes: bool=True):
    """Analyse a (separated) guitar track and return dict OR DataFrames."""
    y, sr = librosa.load(audio_path, sr=None)
    print(f"Loaded {audio_path}  ({len(y)/sr:.1f}s, {sr} Hz)")

    # ----- Beat tracking ------------------------------------------------------
    tmp_tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr, units='frames', tightness=400)
    # librosa ≥1.0 may return a 0‑D ndarray; coerce to plain float
    tempo = float(np.atleast_1d(tmp_tempo)[0])
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    step = 60/tempo/4          # 16‑note subdivision
    grid_times = np.arange(beat_times[0], beat_times[-1]+step, step)

    # ----- Onset detection ----------------------------------------------------
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    # ----- Prepare harmonic signal + chroma ----------------------------------
    y_harm, _ = librosa.effects.hpss(y)
    chroma = librosa.feature.chroma_cqt(y=y_harm, sr=sr)  # keyword args for librosa>=1.0

    # ----- Directions and velocities -----------------------------------------
    directions = spectral_centroid_direction(y, sr, onset_frames)
    rms = librosa.feature.rms(y=y, frame_length=2048, hop_length=512)[0]

    # ----- Chords (per beat) --------------------------------------------------
    beat_chords = chord_sequence_madmom(audio_path, sr)

    # ----- Build Strum list ---------------------------------------------------
    strums: List[Strum] = []
    for i, (t, f) in enumerate(zip(onset_times, onset_frames)):
        gidx = int(np.argmin(np.abs(grid_times - t)))
        bar_idx = gidx // 16
        sub16   = gidx % 16
        vel = float(rms[min(len(rms)-1, f)])
        frame_for_chroma = librosa.time_to_frames(t, sr=sr)
        kind, label = label_note_or_chord(chroma, frame_for_chroma)
        strums.append(Strum(time=float(t), bar=bar_idx+1, sub_16=sub16,
                            direction=directions[i], velocity=vel,
                            kind=kind, label=label))

    # ----- Summarise bars -----------------------------------------------------
    bars: Dict[int, BarSummary] = {}
    for s in strums:
        if s.bar not in bars:
            bars[s.bar] = BarSummary(bar=s.bar,
                                     bit_pattern=['0']*16,
                                     down_up=['-']*16,
                                     mean_vel=0.0,
                                     chords=[])
        b = bars[s.bar]
        b.bit_pattern[s.sub_16] = '1'
        b.down_up[s.sub_16]     = s.direction
        b.mean_vel += s.velocity
        if s.kind == 'CHORD':
            b.chords.append(s.label)
    for b in bars.values():
        hits = b.bit_pattern.count('1')
        b.mean_vel /= max(1, hits)
        b.bit_pattern = ''.join(b.bit_pattern)
        b.down_up    = ' '.join(b.down_up)
        b.chords     = sorted(set(b.chords))

    # ----- Group Sections -----------------------------------------------------
    bar_ordered = [bars[k] for k in sorted(bars)]
    sections: List[Section] = []
    if bar_ordered:
        cur = Section(start_bar=bar_ordered[0].bar, end_bar=bar_ordered[0].bar,
                       pattern_bits=bar_ordered[0].bit_pattern,
                       chords=bar_ordered[0].chords)
        for b in bar_ordered[1:]:
            d = levenshtein(b.bit_pattern, cur.pattern_bits)
            if d <= 2:
                cur.end_bar = b.bar
                cur.chords  = sorted(set(cur.chords + b.chords))
            else:
                sections.append(cur)
                cur = Section(start_bar=b.bar, end_bar=b.bar,
                               pattern_bits=b.bit_pattern, chords=b.chords)
        sections.append(cur)

    # ----- Return -------------------------------------------------------------
    if return_dataframes:
        df_strums = pd.DataFrame([asdict(s) for s in strums])
        df_bars   = pd.DataFrame([asdict(b) for b in bar_ordered])
        df_secs   = pd.DataFrame([asdict(s) for s in sections])
        return {
            'tempo_bpm': tempo,
            'strums': df_strums,
            'bars': df_bars,
            'sections': df_secs
        }
    else:
        return {
            'tempo_bpm': float(tempo),
            'strums': [asdict(s) for s in strums],
            'bars': [asdict(b) for b in bar_ordered],
            'sections': [asdict(s) for s in sections]
        }

# -----------------------------------------------------------------------------
# Notebook helper -------------------------------------------------------------

def run_in_notebook(audio_path: str):
    """Analyse `audio_path` and pretty‑print DataFrames (Jupyter only)."""
    data = analyse_audio(audio_path, return_dataframes=True)
    print(f"Tempo ≈ {data['tempo_bpm']:.1f} BPM")
    from IPython.display import display
    print("\nStrums (first 10):")
    display(data['strums'].head(10))
    print("\nBar summaries:")
    display(data['bars'].head())
    print("\nSections:")
    display(data['sections'])
    return data


In [29]:
audio_path = 'outputs/JeffBuckley-LoverYouShouldveComeOverAudio/stage2_guitar_enhanced_cut.wav'
data = run_in_notebook(audio_path)

Loaded outputs/JeffBuckley-LoverYouShouldveComeOverAudio/stage2_guitar_enhanced_cut.wav  (55.0s, 44100 Hz)
Tempo ≈ 120.2 BPM

Strums (first 10):


/var/folders/sw/2zngpy_n771gjrnyvn_x3mlw0000gn/T/ipykernel_18655/3356485487.py:86: UserWarning:

madmom not available – chord labels will be rough.



,time,bar,sub_16,direction,velocity,kind,label
0,0.081270,1,0,U,0.033050,CHORD,Dmaj
1,0.592109,1,0,U,0.031410,NOTE,D
2,0.893968,1,0,U,0.016532,CHORD,Dmaj
3,1.091338,1,0,U,0.026228,CHORD,D
4,1.404807,1,2,U,0.015962,CHORD,Dmaj
5,1.439637,1,3,D,0.017902,CHORD,D
6,1.602177,1,4,D,0.030112,CHORD,D
7,2.078186,1,8,U,0.025283,CHORD,D
8,2.391655,1,10,U,0.017051,CHORD,D
9,2.879274,1,14,U,0.013744,CHORD,D



Bar summaries:


,bar,bit_pattern,down_up,mean_vel,chords
0,1,1011100010100010,U - U D D - - - U - U - - - U -,0.032468,"[D, Dmaj]"
1,2,1000101010101000,U - - - U - D - U - U - D - - -,0.040871,"[C, D]"
2,3,1010101110001010,U - U - D - U D U - - - U - U -,0.019006,"[C, E]"
3,4,1010100010101001,U - D - U - - - D - U - D - - U,0.027077,"[D, E, Emin]"
4,5,1010100110111000,U - U - U - - D U - U D U - - -,0.023267,"[Cmaj, D, E, Emin]"



Sections:


,start_bar,end_bar,pattern_bits,chords
0,1,1,1011100010100010,"[D, Dmaj]"
1,2,2,1000101010101000,"[C, D]"
2,3,3,1010101110001010,"[C, E]"
3,4,4,1010100010101001,"[D, E, Emin]"
4,5,5,1010100110111000,"[Cmaj, D, E, Emin]"
5,6,6,1010001010001001,"[D, Dmaj]"
6,7,7,1001100010110010,"[D, Dmaj]"
7,8,8,1000101110101000,"[C, Cmaj, D]"
8,9,9,1010101110001011,"[C, E]"
9,10,10,1010100010011001,"[C, E, Emin]"


In [30]:
import json
import pandas as pd
import numpy as np

# Function to convert pandas DataFrames and NumPy arrays to JSON-serializable types
def convert_to_serializable(obj):
    if isinstance(obj, pd.DataFrame):
        return obj.to_dict(orient='records')  # Convert DataFrame to list of dictionaries
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj) if isinstance(obj, np.floating) else int(obj)
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list) or isinstance(obj, tuple):
        return [convert_to_serializable(i) for i in obj]
    else:
        return obj

# Convert the data and save to JSON
serializable_data = convert_to_serializable(data)

# Save to file
with open('outputs/JeffBuckley-LoverYouShouldveComeOverAudio/guitar_data.json', 'w') as f:
    json.dump(serializable_data, f, indent=4)